# Двумерная интерполяция сплайнами

 #### Двумерная интерполяция сплайнами (x(t), y(t)) на равномерной сетке по t с графическим интерфейсом.
 #### Реализация на PyQt5.
 
 Некоторые детали:
 - интерполяция достраивается после нажатия мышкой на графическое полотно
 - с очередным добавлением точки в массив **t** добавляется число, на 10 превышающее предыдущее
 - при вычислении интерполированных данных массив **t_new** разбивается на большее число точек чем **t**
 - интерполяция проходит для функций **x(t)** и **y(t)** раздельно

In [1]:
%matplotlib notebook

In [2]:
import sys
import numpy as np

from PyQt5 import QtCore, QtWidgets, QtGui

import matplotlib
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


class Canvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        fig.suptitle('Двумерная интерполяция на PyQt')
        super(Canvas, self).__init__(fig)


class MainWindow(QtWidgets.QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        self.canvas = Canvas(self, width=5, height=4, dpi=100)
        self.setCentralWidget(self.canvas)
        self.canvas.mpl_connect('button_press_event', self.mouse_click)

        self.xdata = np.array([])       # эти массивы хранять только точки, добавленные мышкой
        self.ydata = np.array([])
        self.tdata = np.array([])
        self.cnt = 0                    # подсчет количества нажатий

        self.updated_xdata = np.array([]) # эти массивы хранят интерполированные значения
        self.updated_ydata = np.array([])

        self.update_plot()

        toolbar = NavigationToolbar(self.canvas, self) # Следующий код предназначен для панели инструментов

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(toolbar)
        layout.addWidget(self.canvas)

        widget = QtWidgets.QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

        self.show()



    def update_plot(self):                  # Обновление графика
        self.canvas.axes.cla()              # Сначала очищаем
        self.canvas.axes.grid(True)
        self.canvas.axes.axis([-10, 10, -10, 10])
        self.canvas.axes.plot(self.xdata, self.ydata, 'rx--', label='Линейная', linewidth=0.2)
        self.canvas.axes.plot(self.updated_xdata, self.updated_ydata, 'b', label='Сплайновая')
        self.canvas.axes.set(xlabel='X(t)', ylabel='Y(t)')
        self.canvas.axes.legend()
        self.canvas.draw()



    def mouse_click(self, event):               # Обработка нажатия мышкой на полотно
        self.cnt = self.cnt+1
        ix, iy = event.xdata, event.ydata   # Считывание координат точки нажатия
        print('x=%f,y=%f'%(ix, iy))
        self.xdata = np.append(self.xdata, [ix])   # Добавление координат новой точки
        self.ydata = np.append(self.ydata, [iy])
        self.tdata = np.append(self.tdata, [0+10*(self.cnt-1)])

        if len(self.xdata) > 1:        # Функция интерполирования начинает работу имея 2 точки

            t_new = np.linspace(self.tdata.min(), self.tdata.max(), 1000)
            x_new = evaluate(t_new, self.tdata, self.xdata)
            y_new = evaluate(t_new, self.tdata, self.ydata)

            self.updated_xdata = x_new.copy()
            self.updated_ydata = y_new.copy()

        self.update_plot()




######################  Следующий код предназначен для вычисления интерполированных данных ################################

def sweep (a, b, c, f):       # В ходе отыскания коэффициентов многочленов будет нужна
    n = f.size                #      функция для решения СЛАУ методом прогонки

    alpha = [0] * (n+1)
    betta = [0] * (n+1)
    x = [0] * (n+1)
    a[0] = 0
    c[n-1] = 0
    alpha[0] = 0
    betta[0] = 0

    for i in range(n):
        d = a[i] * alpha[i] + b[i]
        alpha[i+1] = -c[i] / d
        betta[i+1] = (f[i] - a[i] * betta[i]) / d

    x[n] = 0
    for i in range(n-1, -1, -1):
        x[i] = alpha[i+1] * x[i+1] + betta[i+1]

    x = x[:-1]
    return x



def generateSpline(x, y):           # Коэффициенты строим здесь
    n = x.shape[0] - 1
    h = (x[n] - x[0]) / n

    a = np.array([0] + [1] * (n - 1) + [0])
    b = np.array([1] + [4] * (n - 1) + [1])
    c = np.array([0] + [1] * (n - 1) + [0])
    f = np.zeros(n + 1)

    for i in range(1, n):
        f[i] = 3 * (y[i-1] - 2 * y[i] + y[i+1]) / (h**2)
    s = sweep(a, b, c, f)

    A = np.zeros(n+1)
    B = np.zeros(n+1)
    C = np.zeros(n+1)
    D = np.zeros(n+1)

    for i in range(n):
        B[i] = s[i]

    for i in range(n):
        A[i] = (B[i+1] - B[i]) / (3 * h)
        C[i] = (y[i+1] - y[i]) / h - (B[i+1] + 2 * B[i]) * h / 3
        D[i] = y[i]

    return A, B, C, D


def find_value(dest, x, y, A, B, C, D):         # Значение функции для очередной точки
    n = x.shape[0] - 1

    if dest < x[0] or dest > x[n]:
        print('Выход из промежутка интерполирования')

    if dest == x[n]:
        return y[n]

    for i in range(n):
        if dest >= x[i] and dest < x[i+1]:
            index = i

    P = A[index] * (dest - x[index]) ** 3
    P += B[index] * (dest - x[index]) ** 2
    P += C[index] * (dest - x[index]) + D[index]

    return P


def evaluate(x_new, x, y):  		   # Вычислить весь набор интерполированных данных
    A, B, C, D = generateSpline(x, y)

    f = np.zeros(len(x_new))

    for i in range(len(x_new)):
        f[i] = find_value(x_new[i], x, y, A, B, C, D)

    return f



######################################################

app = QtWidgets.QApplication(sys.argv)
w = MainWindow()
app.exec_()

x=-7.750550,y=-6.130640
x=-4.247152,y=1.244200
x=-3.289940,y=6.025690
x=-3.921700,y=7.849140
x=-4.610893,y=7.565492
x=-4.821480,y=5.742043
x=-1.471236,y=4.607452
x=2.568201,y=4.404846
x=4.961233,y=4.283283
x=4.922944,y=-0.579249
x=1.457835,y=-3.739895
x=-0.992629,y=-3.537290
x=2.472480,y=-0.903418
x=3.908299,y=-5.279697
x=6.492773,y=-7.143668


0